# Thai N-NER: Thai Nested Named Entity Recognition

This demo notebook provides a tutorial on using Thai N-NER, with references from [Thai N-NER](https://medium.com/airesearch-in-th/thai-n-ner-thai-nested-named-entity-recognition-1969f8fe91f0)

Learn more about Thai N-NER here : [Thai N-NER](https://medium.com/airesearch-in-th/thai-n-ner-thai-nested-named-entity-recognition-1969f8fe91f0)

## 1. Setup and Preprocessing

In [1]:
!pip install seqeval
!pip install pythainlp
!pip install transformers==4.29.2
!pip install sentencepiece
!pip install gdown
!pip install thai_nner
!pip install protobuf==3.20.3


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached transformers-4.29.2-py3-none-any.whl.metadata (112 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... -^C
anceled

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Model checkpoints

> Thai N-NER provides necessary resources, including models, datasets, and pre-trained language models, available here : [Thai N-NER (resources)](https://drive.google.com/drive/folders/1Dy-360iZ9hIA-xA0yizSwmpM8sx6rrjJ?usp=sharing)

To utilize this, please follow these steps::

1. Add the Shared Folder [Thai N-NER (resources)](https://drive.google.com/drive/folders/1Dy-360iZ9hIA-xA0yizSwmpM8sx6rrjJ?usp=sharing)  to Your Google Drive.
* first open the shared folder link in your web browser
* Click the folder named "thai-nner" at the top of the page.
* In the menu bar, click "Organize", then click "Add shortcut" to Drive (you may see an icon that looks like a Drive logo with a plus sign)
* Select "My Drive"


In [2]:
# Clone github
!git clone https://github.com/vistec-AI/Thai-NNER.git
%cd /content/Thai-NNER

Cloning into 'Thai-NNER'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 274 (delta 16), reused 5 (delta 2), pack-reused 239 (from 1)
Receiving objects: 100% (274/274), 5.23 MiB | 772.00 KiB/s, done.
Resolving deltas: 100% (134/134), done.
[Errno 2] No such file or directory: '/content/Thai-NNER'
/Users/idhibhatpankam/Code/courses/NLP-SYS/codes/L4_Token_Classification


/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


# Mount your drive to Google Colab.

In [2]:
# Load data
from google.colab import drive
drive.mount('/content/drive/')

# Create symbolic links
!ln -s "/content/drive/MyDrive/thai-nner/lm" ./data/lm
!ln -s "/content/drive/MyDrive/thai-nner/checkpoints" ./data/checkpoints

ModuleNotFoundError: No module named 'google.colab'

# Inference

In [8]:
import json
import torch
import argparse
from tqdm import tqdm
from tabulate import tabulate

from ThaiNNER.utils.unique import unique
import ThaiNNER.model.loss as module_loss
import ThaiNNER.model as module_arch
import ThaiNNER.model.metric as module_metric
from ThaiNNER.parse_config import ConfigParser
import ThaiNNER.data_loader.data_loaders as module_data

PAD = '<pad>'

ModuleNotFoundError: No module named 'logger'

In [ ]:
resume = 'data/checkpoints/1102_151935/checkpoint.pth'

In [ ]:
args = argparse.ArgumentParser(description='PyTorch Template')
args.add_argument('-c', '--config', default=None, type=str, help='config file path (default: None)')
args.add_argument('-r', '--resume', default=f"{resume}", type=str, help='path to latest checkpoint (default: None)')
args.add_argument('-d', '--device', default=None, type=str, help='indices of GPUs to enable (default: all)')
args.add_argument('-f', '--file', default=None, type=str, help='Error')
config = ConfigParser.from_args(args)
logger = config.get_logger('test')

# build model architecturea
model = config.init_obj('arch', module_arch)

# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]

logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume)
state_dict = checkpoint['state_dict']

if config['n_gpu'] > 1:
    model = torch.nn.DataParallel(model)

model.load_state_dict(state_dict)
layers_train = config._config['trainer']['layers_train']

# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

total_loss = 0.0
total_metrics = torch.zeros(len(metric_fns))
# logger.info(model)

In [ ]:
# Loading only few testing examples.
config.config['data_loader']['args']['sample_data'] = True

data_loader = config.init_obj('data_loader', module_data)
test_data_loader = data_loader.get_test()

> Now, Let's try using the pre-trained Thai N-NER model checkpoint to perform inference and predict NE tags.

In [ ]:
from utils.prediction import predict, get_dict_prediction, show



text = " วันนี้วันที่ 27 มกราคม 2568 เป็นวันที่อากาศดีมาก "


tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

In [ ]:
text = "คณะกรรมการ 40 ปี 14 ตุลาเพื่อประชาธิปไตรสมบูรณ์"
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

In [ ]:
text = " วันที่ 18 มกราคม 2568 เมื่อเวลา 11.15 น. ที่จ.นครพนม นายทักษิณ ชินวัตร อดีตนายกฯ ให้สัมภาษณ์กรณีนายชาดา ไทยเศรษฐ์ อดีต รมช.มหาดไทย เซ็นคำสั่งเพิกถอนที่ดินสนามกอล์ฟอัลไพน์ กลับคืนเป็นที่ธรณีสงฆ์ ก่อนหมดวาระเพียงไม่กี่วัน "
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];

In [ ]:
text = " สธ.กางตัวเลขเบื้องต้นคนป่วยจากปัญหาฝุ่น PM2.5 แค่ 3 สัปดาห์ของเดือน ม.ค.พุ่ง 144,000 คนส่วนใหญ่ผิวหนัง ตาอักเสบ โรคหืด พบ 5 จังหวัดค่าฝุ่นเกิน 75 มคก.ต่อ ลบ.ม.ต่อเนื่องเกิน 3 ในระดับสีแดง "
tokens, out = predict(model, text, data_loader, config)
tokens = [tk for tk in tokens if tk!=PAD]
print("|".join(tokens), "\n")
[show(x) for x in out];